**Задание 1: Векторизация текста с использованием Мешка слов (BoW) и TF-IDF**

1. Создайте корпус из минимум 10 текстовых документов (можно взять новостные статьи, отзывы на товары, статьи по одной тематике).
2. Реализуйте предобработку текста, включающую:
* Приведение к нижнему регистру
* Токенизацию
* Удаление стоп-слов и пунктуации
* Лемматизацию/стемминг
3. Реализуйте модель Мешка слов (BoW) с использованием CountVectorizer из scikit-learn.
4. Реализуйте модель TF-IDF с использованием TfidfVectorizer из scikit-learn.
5. Найдите 10 самых значимых терминов для каждого документа по обоим подходам и сравните результаты.
6. Визуализируйте сходство документов с помощью метрики косинусного расстояния для обоих подходов.
7. Прокомментируйте разницу в результатах между BoW и TF-IDF

In [1]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 21.3 MB/s eta 0:00:00


In [2]:
import string
import re
import nltk
import spacy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt_tab')

# Загрузка моделей spaCy
nlp_en = spacy.load("en_core_web_sm")

# Загрузка стоп-слов
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
original_documents = ["""Trump’s E.P.A. Has Put a Value on Human Life: Zero Dollars. The Environmental Protection Agency has stopped estimating the dollar value of lives saved in the cost-benefit analyses for new pollution rules.""",
                      """The Cheating Scandal That Rocked Norway Before the Winter Olympics. A video posted anonymously to YouTube revealed that the country manipulated the suits of the team’s top ski jumpers.""",
                      """Halligan Leaves as U.S. Attorney After Mounting Pressure From Judges. For weeks, judges have pressed Ms. Halligan to explain why she continues to identify herself in court filings as the U.S. attorney, despite a ruling in November that she was unlawfully appointed to the job.""",
                      """She Listened to Women’s Pain. Then She Transformed How It Was Treated. Ghada Hatem-Gantzer, a French doctor, founded a one-stop shop to treat female victims of violence. Now, her revolutionary template is used across France.""",
                      """Supreme Court Considers Trump’s Attempt to Fire Fed Governor Lisa Cook. The justices deferred a decision on the president’s efforts to oust Cook, agreeing to hear arguments on Wednesday instead.""",
                      """Taiwan’s $40 Billion Military Spending Plan Is Stalled by Political Impasse. Taiwan’s domestic gridlock is revealing a deep-seated fracture over how the island should defend itself and how much it can depend on the United States.""",
                      """‘Marty Supreme’ Has a Lot to Say About Being Jewish in America. “Marty Supreme” is one of the great Jewish movies, an unapologetic depiction of the experience in all of its complications, a critic writes.""",
                      """Hidden in a Stream, New Clues About the Cause of the Spain Rail Crash. Officials said they had located a previously unreported train undercarriage near the site of a deadly train crash in Spain. Experts said the finding could help investigators.""",
                      """An Anxious Japan Restarts the World’s Biggest Nuclear Plant. Japan is reviving nuclear power, balancing the need for more clean energy against the lingering trauma of the 2011 Fukushima Daiichi disaster.""",
                      """The Beckhams Are Fighting. Here’s What’s Going On. Victoria and David Beckham have not responded directly since their son Brooklyn posted a series of accusations on Instagram against his parents.""",
                      """South Korean Ex-Premier Gets 23 Years in Prison for Role in Martial Law. Han Duck-soo was convicted of playing a key role in former President Yoon Suk Yeol’s imposition of martial law, which a court said was an insurrection.""",
                      """Killer of Shinzo Abe, Former Leader of Japan, Gets Life in Prison. Tetsuya Yamagami had admitted to shooting Mr. Abe, Japan’s longest-serving prime minister, but his case divided the country."""]


In [4]:
# Предобработка текста
def normalize_text(text):
    new_text = []
    for sentence in text:
        sentence = sentence.lower()
        sentence = re.sub(r'[^\w\s]', ' ', sentence)
        # Нужно убрать числа так как они часто попадались в BOW
        sentence = re.sub(r'[0-9]+', ' ', sentence)
        tokens = sentence.split()
        filtered_tokens = [token for token in tokens if token not in russian_stopwords]
        sentence = ' '.join(filtered_tokens)

        # Лемматизация
        doc = nlp_en(sentence)
        lemmas = [token.lemma_ for token in doc if token.is_alpha]
        sentence = ' '.join(lemmas)
        new_text.append(sentence)

    return new_text

documents = normalize_text(original_documents)

print("Предобработанные документы:")
for i, doc in enumerate(documents, 1):
    print(f"Документ {i}: {doc}")

Предобработанные документы:
Документ 1: trump e p put value human life zero dollar environmental protection agency stop estimate dollar value life save cost benefit analyse new pollution rule
Документ 2: cheat scandal rock norway winter olympics video post anonymously youtube reveal country manipulate suit team top ski jumper
Документ 3: halligan leave u attorney mount pressure judge week judge press ms halligan explain continues identify court filing u attorney despite rule november unlawfully appoint job
Документ 4: listen woman pain transform treat ghada hatem gantzer french doctor found one stop shop treat female victim violence revolutionary template use across france
Документ 5: supreme court consider trump attempt fire feed governor lisa cook justice defer decision president effort oust cook agree hear argument wednesday instead
Документ 6: taiwan billion military spending plan stall political impasse taiwan domestic gridlock reveal deep seat fracture island defend much depend u

In [9]:
from sklearn.feature_extraction.text import CountVectorizer  # Для создания Bag of Words

In [10]:
# Создание векторизатора
count_vectorizer = CountVectorizer()

# Обучение векторизатора и преобразование документов
bow_matrix = count_vectorizer.fit_transform(documents)

# Получение списка фичей (слов)
feature_names = count_vectorizer.get_feature_names_out()

# Преобразование разреженной матрицы в плотную для наглядности
bow_df = pd.DataFrame(
    bow_matrix.toarray(),
    columns=feature_names,
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

# Вывод матрицы Bag of Words
print("Матрица Bag of Words:")
print(bow_df)

Матрица Bag of Words:
             abe  accusation  across  admit  agency  agree  america  analyse  \
Документ 1     0           0       0      0       1      0        0        1   
Документ 2     0           0       0      0       0      0        0        0   
Документ 3     0           0       0      0       0      0        0        0   
Документ 4     0           0       1      0       0      0        0        0   
Документ 5     0           0       0      0       0      1        0        0   
Документ 6     0           0       0      0       0      0        0        0   
Документ 7     0           0       0      0       0      0        1        0   
Документ 8     0           0       0      0       0      0        0        0   
Документ 9     0           0       0      0       0      0        0        0   
Документ 10    0           1       0      0       0      0        0        0   
Документ 11    0           0       0      0       0      0        0        0   
Документ 12    2  

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
# Создание TF-IDF векторизатора
tfidf_vectorizer = TfidfVectorizer()

# Обучение векторизатора и преобразование документов
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Преобразование в DataFrame
tfidf_df = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=tfidf_vectorizer.get_feature_names_out(),
    index=[f'Документ {i+1}' for i in range(len(documents))]
)

# Вывод матрицы TF-IDF
print("Матрица TF-IDF:")
print(tfidf_df)

Матрица TF-IDF:
                  abe  accusation    across     admit    agency     agree  \
Документ 1   0.000000     0.00000  0.000000  0.000000  0.196493  0.000000   
Документ 2   0.000000     0.00000  0.000000  0.000000  0.000000  0.000000   
Документ 3   0.000000     0.00000  0.000000  0.000000  0.000000  0.000000   
Документ 4   0.000000     0.00000  0.202133  0.000000  0.000000  0.000000   
Документ 5   0.000000     0.00000  0.000000  0.000000  0.000000  0.209481   
Документ 6   0.000000     0.00000  0.000000  0.000000  0.000000  0.000000   
Документ 7   0.000000     0.00000  0.000000  0.000000  0.000000  0.000000   
Документ 8   0.000000     0.00000  0.000000  0.000000  0.000000  0.000000   
Документ 9   0.000000     0.00000  0.000000  0.000000  0.000000  0.000000   
Документ 10  0.000000     0.23744  0.000000  0.000000  0.000000  0.000000   
Документ 11  0.000000     0.00000  0.000000  0.000000  0.000000  0.000000   
Документ 12  0.402928     0.00000  0.000000  0.201464  0.000

In [13]:
bow_sum = np.sum(bow_matrix.toarray(), axis=0)

word_bow_dict = dict(zip(feature_names, bow_sum))

top_bow_words = sorted(word_bow_dict.items(), key=lambda x: x[1], reverse=True)[:10]

print("=== Топ-10 слов по всей коллекции ===\n")
print("Топ-10 слов по BOW (самые частые в коллекции):")
for word, count in top_bow_words:
    print(f"- {word}: {count} раз")

tfidf_sum = np.sum(tfidf_matrix.toarray(), axis=0)

word_tfidf_dict = dict(zip(feature_names, tfidf_sum))

top_tfidf_words = sorted(word_tfidf_dict.items(), key=lambda x: x[1], reverse=True)[:10]

print("\nТоп-10 слов по TF-IDF (с наибольшим весом по всей коллекции):")
for word, score in top_tfidf_words:
    print(f"- {word}: {score:.4f}")

print("\nСравнение топ-10 слов:")
bow_words = [word for word, _ in top_bow_words]
tfidf_words = [word for word, _ in top_tfidf_words]

common_words = set(bow_words) & set(tfidf_words)
bow_only = set(bow_words) - common_words
tfidf_only = set(tfidf_words) - common_words

if common_words:
    print(f"Общие слова в обоих топ-10: {', '.join(common_words)}")
if bow_only:
    print(f"Только в BOW топ-10: {', '.join(bow_only)}")
if tfidf_only:
    print(f"Только в TF-IDF топ-10: {', '.join(tfidf_only)}")

=== Топ-10 слов по всей коллекции ===

Топ-10 слов по BOW (самые частые в коллекции):
- japan: 4 раз
- say: 4 раз
- court: 3 раз
- life: 3 раз
- supreme: 3 раз
- abe: 2 раз
- attorney: 2 раз
- beckham: 2 раз
- cook: 2 раз
- country: 2 раз

Топ-10 слов по TF-IDF (с наибольшим весом по всей коллекции):
- japan: 0.7046
- say: 0.5586
- supreme: 0.5439
- life: 0.5105
- beckham: 0.4749
- court: 0.4354
- jewish: 0.4239
- marty: 0.4239
- taiwan: 0.4194
- cook: 0.4190

Сравнение топ-10 слов:
Общие слова в обоих топ-10: life, japan, beckham, court, supreme, cook, say
Только в BOW топ-10: attorney, country, abe
Только в TF-IDF топ-10: jewish, taiwan, marty


**Задание 2: Морфологическая разметка текста**

1. Возьмите отрывок текста (минимум 300 слов) на русском и английском языке.
2. Для русского языка используйте PyMorphy2 или PyMorphy3, для английского - NLTK или SpaCy для проведения морфологического анализа.
3. Выполните следующие операции:
* Определите части речи для каждого слова в тексте
* Для существительных определите падеж, род и число
* Для глаголов определите время, лицо и число
* Создайте частотный словарь частей речи в тексте
4. Разработайте функцию, которая будет автоматически изменять текст, заменяя все существительные на их форму множественного числа (где возможно).
5. Результаты морфологического анализа должны быть представлены в виде таблицы.
6. Оцените и прокомментируйте точность определения морфологических характеристик

In [14]:
english_text = """‘Marty Supreme’ Has a Lot to Say About Being Jewish in America.
The film’s unapologetic depiction of the experience in all of its complications rejects the idea that such characters have to suffer. “Marty Supreme” is full of shocking moments. There are unexpected bursts of violence. A bathtub falls through the ceiling. But perhaps the most shocking is a simple line of dialogue.
The table-tennis phenom Marty Mauser (Timothée Chalamet) is being interviewed by two stodgy reporters at the Ritz in London. Asked about a Hungarian rival in the British Open, Marty says he’s going to do to that player what “Auschwitz couldn’t.” He continues, with a deadpan look in his eyes, “I’m going to finish the job.”
I’ve seen “Marty Supreme” three times, and every time Marty utters those words there’s been a palpable shift in the audience: nervous titters or stunned gasps.
Marty quickly backtracks: He’s allowed to say that, he contends, because he’s Jewish. But the moment is so abrasive, it sticks in your craw. It’s also crucial to understanding the movie’s
relationship to Marty’s Jewishness, an aspect of his persona that has provoked a lively discussion online. For some viewers, the portrayal of a ruthlessly ambitious American Jew borders on the antisemitic. For others, myself included, “Marty Supreme” is one of the great Jewish movies, an unapologetic depiction of the Jewish
American experience in all of its complications. The director Josh Safdie and his co-writer, Ronald Bronstein, have an abiding affection for bombastic Jewish heroes who ruffle feathers. Their previous collaboration, “Uncut Gems,” which Safdie directed alongside his brother, Benny, was set in Manhattan’s Diamond District and
followed a compulsive gambler (Adam Sandler) who can’t get through a Passover Seder without some mishegas. The brand of Jewish man who interests Safdie is not a model minority. The director is drawn to brash hustlers who make decisions that get themselves and the people around them into loads of trouble. And yet these
characters are written with love and a deep understanding of the evolution of Jewish New York.
"""
russian_text = """Когда кавалеристы проходили через деревню Бережки, немецкий снаряд разорвался на околице и ранил в ногу вороного коня. Командир оставил раненого коня в деревне, а отряд ушёл дальше, пыля и позванивая удилами, – ушёл, закатился за рощи, за холмы, где ветер качал спелую рожь.
Коня взял к себе мельник Панкрат. Мельница давно не работала, но мучная пыль навеки въелась в Панкрата. Она лежала серой коркой на его ватнике и картузе. Из-под картуза посматривали на всех быстрые глаза мельника. Панкрат был скорый на работу, сердитый старик, и ребята считали его колдуном.
Панкрат вылечил коня. Конь остался при мельнице и терпеливо возил глину, навоз и жерди – помогал Панкрату чинить плотину.
Панкрату трудно было прокормить коня, и конь начал ходить по дворам побираться. Постоит, пофыркает, постучит мордой в калитку, и, глядишь, ему вынесут свекольной ботвы, или чёрствого хлеба, или, случалось даже, сладкую морковку. По деревне говорили, что конь ничей, а вернее – общественный, и
каждый считал своей обязанностью его покормить. К тому же конь – раненый, пострадал от врага.
Жил в Бережках со своей бабкой мальчик Филька, по прозвищу «Ну Тебя». Филька был молчаливый, недоверчивый, и любимым его выражением было: «Да ну тебя!» Предлагал ли ему соседский мальчишка походить на ходулях или поискать позеленевшие патроны, Филька отвечал сердитым басом: «Да ну тебя! Ищи сам!»
Когда бабка выговаривала ему за неласковость, Филька отворачивался и бормотал: «Да ну тебя! Надоела!»
Зима в этот год стояла тёплая. В воздухе висел дым. Снег выпадал и тотчас таял. Мокрые вороны садились на печные трубы, чтобы обсохнуть, толкались, каркали друг на друга. Около мельничного лотка вода не замерзала, а стояла чёрная, тихая, и в ней кружились льдинки.
Панкрат починил к тому времени мельницу и собирался молоть хлеб, – хозяйки жаловались, что мука кончается, осталось у каждой на два-три дня, а зерно лежит немолотое.
В один из таких тёплых серых дней раненый конь постучал мордой в калитку к Филькиной бабке. Бабки не было дома, а Филька сидел за столом и жевал кусок хлеба, круто посыпанный солью.
Филька нехотя встал, вышел за калитку. Конь переступил с ноги на ногу и потянулся к хлебу. «Да ну тебя! Дьявол!» – крикнул Филька и наотмашь ударил коня по губам. Конь отшатнулся, замотал головой, а Филька закинул хлеб далеко в рыхлый снег и закричал:
– На вас не напасёшься, на христорадников! Вон твой хлеб! Иди копай его мордой из-под снега! Иди копай!"""

In [15]:
!pip install nltk spacy pymorphy3
!pip install inflect

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import Counter
from nltk.tokenize.treebank import TreebankWordDetokenizer
import nltk
import string
import pymorphy3
import inflect

nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger_eng')

import spacy
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 67.5 MB/s eta 0:00:00


[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [17]:
# Распределить на части речи англ слова
text_tok = word_tokenize(english_text)
tagged = nltk.pos_tag(text_tok)
list_of_tagged = []

for elem in tagged:
    tag_tog = '_'.join(elem)
    list_of_tagged.append(tag_tog)

print(list_of_tagged)

['‘_JJ', 'Marty_NNP', 'Supreme_NNP', '’_NNP', 'Has_VBZ', 'a_DT', 'Lot_NN', 'to_TO', 'Say_VB', 'About_IN', 'Being_VBG', 'Jewish_JJ', 'in_IN', 'America_NNP', '._.', 'The_DT', 'film_NN', '’_NNP', 's_VBZ', 'unapologetic_JJ', 'depiction_NN', 'of_IN', 'the_DT', 'experience_NN', 'in_IN', 'all_DT', 'of_IN', 'its_PRP$', 'complications_NNS', 'rejects_VBZ', 'the_DT', 'idea_NN', 'that_IN', 'such_JJ', 'characters_NNS', 'have_VBP', 'to_TO', 'suffer_VB', '._.', '“_NNP', 'Marty_NNP', 'Supreme_NNP', '”_NNP', 'is_VBZ', 'full_JJ', 'of_IN', 'shocking_VBG', 'moments_NNS', '._.', 'There_EX', 'are_VBP', 'unexpected_JJ', 'bursts_NNS', 'of_IN', 'violence_NN', '._.', 'A_DT', 'bathtub_NN', 'falls_VBZ', 'through_IN', 'the_DT', 'ceiling_NN', '._.', 'But_CC', 'perhaps_RB', 'the_DT', 'most_RBS', 'shocking_JJ', 'is_VBZ', 'a_DT', 'simple_JJ', 'line_NN', 'of_IN', 'dialogue_NN', '._.', 'The_DT', 'table-tennis_JJ', 'phenom_NN', 'Marty_NNP', 'Mauser_NNP', '(_(', 'Timothée_NNP', 'Chalamet_NNP', ')_)', 'is_VBZ', 'being_VBG'

In [18]:
# Распределить на части речи русские слова
morph = pymorphy3.MorphAnalyzer()
text_tok = word_tokenize(russian_text)
rus_list_of_tagged = []

for word in text_tok:
    parses = morph.parse(word)
    best_parse = parses[0]
    pos = best_parse.tag.POS
    _case = best_parse.tag.case
    gender = best_parse.tag.gender
    number = best_parse.tag.number
    person = best_parse.tag.person
    tense = best_parse.tag.tense
    if pos == 'NOUN':
        rus_list_of_tagged.append(f'{word}_{pos}_{_case}_{gender}_{number}')
    elif pos == 'VERB':
        rus_list_of_tagged.append(f'{word}_{pos}_{person}_{tense}_{number}')
    else:
        rus_list_of_tagged.append(f'{word}_{pos}')

print(rus_list_of_tagged)

['Когда_CONJ', 'кавалеристы_NOUN_nomn_masc_plur', 'проходили_VERB_None_past_plur', 'через_PREP', 'деревню_NOUN_accs_femn_sing', 'Бережки_NOUN_nomn_masc_plur', ',_None', 'немецкий_ADJF', 'снаряд_NOUN_nomn_masc_sing', 'разорвался_VERB_None_past_sing', 'на_PREP', 'околице_NOUN_datv_femn_sing', 'и_CONJ', 'ранил_VERB_None_past_sing', 'в_PREP', 'ногу_NOUN_accs_femn_sing', 'вороного_ADJF', 'коня_NOUN_gent_masc_sing', '._None', 'Командир_NOUN_nomn_masc_sing', 'оставил_VERB_None_past_sing', 'раненого_ADJF', 'коня_NOUN_gent_masc_sing', 'в_PREP', 'деревне_NOUN_loct_femn_sing', ',_None', 'а_CONJ', 'отряд_NOUN_accs_masc_sing', 'ушёл_VERB_None_past_sing', 'дальше_COMP', ',_None', 'пыля_GRND', 'и_CONJ', 'позванивая_GRND', 'удилами_NOUN_ablt_None_plur', ',_None', '–_None', 'ушёл_VERB_None_past_sing', ',_None', 'закатился_VERB_None_past_sing', 'за_PREP', 'рощи_NOUN_nomn_femn_plur', ',_None', 'за_PREP', 'холмы_NOUN_nomn_masc_plur', ',_None', 'где_ADVB', 'ветер_NOUN_nomn_masc_sing', 'качал_VERB_None_past

In [19]:
# Словарь частей речи англ
tokens = word_tokenize(english_text)
tokens = [t for t in tokens if re.search(r"[A-Za-z0-9]", t)]
tagged = pos_tag(tokens)
pos_counts = Counter(tag for _, tag in tagged)

print(pos_counts.most_common())

[('NN', 49), ('NNP', 44), ('DT', 43), ('IN', 42), ('JJ', 35), ('NNS', 23), ('VBZ', 19), ('PRP', 12), ('VBP', 10), ('TO', 9), ('CC', 9), ('VBN', 9), ('VB', 8), ('RB', 8), ('PRP$', 7), ('VBG', 6), ('VBD', 6), ('WP', 5), ('CD', 3), ('WDT', 3), ('EX', 2), ('RBS', 1), ('MD', 1)]


In [20]:
# Словарь частей речи рус
pos_counts = Counter()

for word in text_tok:

    if all(not ch.isalnum() and ch not in "ёЁ" for ch in word):
        continue
    parse = morph.parse(word)[0]
    pos = parse.tag.POS or "UNKN"
    pos_counts[pos] += 1

print(pos_counts.most_common())

[('NOUN', 122), ('VERB', 75), ('PREP', 46), ('ADJF', 44), ('CONJ', 34), ('NPRO', 17), ('PRCL', 15), ('ADVB', 11), ('INFN', 9), ('COMP', 2), ('GRND', 2), ('PRTF', 2), ('NUMR', 1)]


In [21]:
# Замена ед. сущ. на мн. сущ. АНГЛ
p = inflect.engine()
nlp = spacy.load("en_core_web_sm")

def pluralize_nouns_spacy(text: str) -> str:
    doc = nlp(text)
    out = []

    for tok in doc:
        # NN/NNP = обычно единственное число (NNS/NNPS = уже множественное)
        if tok.tag_ in ("NN", "NNP"):
            pl = p.plural_noun(tok.text) or p.plural(tok.text)

            # сохранение регистра
            if tok.text.isupper():
                pl = pl.upper()
            elif tok.text.istitle():
                pl = pl.title()

            out.append(pl + tok.whitespace_)
        else:
            out.append(tok.text_with_ws)

    return "".join(out)

print(pluralize_nouns_spacy(english_text))

‘Martys Supremes’ Has a Lots to Say About Being Jewish in Americas.
The films’s unapologetic depictions of the experiences in all of its complications rejects the ideas that such characters have to suffer. “Martys Supremes” is full of shocking moments. There are unexpected bursts of violences. A bathtubs falls through the ceilings. But perhaps the most shocking is a simple lines of dialogues.
The tables-tenniss phenoms Martys Mausers (Timothées Chalamets) is being interviewed by two stodgy reporters at the Ritzzes in Londons. Asked about a Hungarian rivals in the Britishes Opens, Martys says he’s going to do to that players what “Auschwitzzes couldn’ts.” He continues, with a deadpans looks in his eyes, “I’m going to finish the jobs.”
I’ve seen “Martys Supremes” three times, and every times Martys utters those words there’s been a palpable shifts in the audiences: nervous titters or stunned gasps.
Martys quickly backtracks: He’s allowed to say that, he contends, because he’s Jewish. But

In [22]:
#  Функция которая меняет ед. сущ. на мн. сущ. РУС
detok = TreebankWordDetokenizer()
def pluralize_nouns_pymorphy() -> str:

    tokens = word_tokenize(russian_text)
    new_tokens = []

    for w in tokens:
        if any(ch.isalpha() for ch in w):
            p = morph.parse(w)[0]
            if p.tag.POS == "NOUN" and p.tag.number == "sing":
                gramm = {"plur"}
                if p.tag.case:
                    gramm.add(p.tag.case)
                if p.tag.animacy:
                    gramm.add(p.tag.animacy)

                inf = p.inflect(gramm)
                if inf:
                    w = inf.word
        new_tokens.append(w)
    return new_tokens

new_text = detok.detokenize(pluralize_nouns_pymorphy())
print(new_text)

Когда кавалеристы проходили через деревни Бережки, немецкий снаряды разорвался на околицам и ранил в ноги вороного коней . командиры оставил раненого коней в деревнях, а отряды ушёл дальше, пыля и позванивая удилами, – ушёл, закатился за рощи, за холмы, где ветры качал спелую ржи . коней взял к себе мельники панкраты . мельницы давно не работала, но мучная пыль навеки въелась в панкратов . Она лежала серой корками на его ватниках и картузах . Из-под картузов посматривали на всех быстрые глаза мельников . панкраты был скорый на работы, сердитый старики, и ребята считали его колдунами . панкраты вылечил коней . кони остался при мельницам и терпеливо возил глины, навозы и жердей – помогал панкратам чинить плотины . панкратам трудно было прокормить коней, и кони начал ходить по дворам побираться . Постоит, пофыркает, постучит мордами в калитки, и, глядишь, ему вынесут свекольной ботвы, или чёрствого хлебов, или, случалось даже, сладкую морковки . По деревнях говорили, что кони ничей, а вер

In [23]:
# Результаты разметки АНГЛ
eng_tokens = word_tokenize(english_text)
eng_tagged = nltk.pos_tag(eng_tokens)  # список (token, tag)

eng_df = pd.DataFrame(eng_tagged, columns=["token", "pos_tag"])


# Результаты разметки РУС
morph = pymorphy3.MorphAnalyzer()
rus_tokens = word_tokenize(russian_text)

rus_rows = []
for word in rus_tokens:
    if all(not ch.isalnum() and ch not in "ёЁ" for ch in word):
        continue

    p = morph.parse(word)[0]
    pos = p.tag.POS or "UNKN"

    row = {
        "token": word,
        "pos_tag": pos,
        "case": p.tag.case,
        "gender": p.tag.gender,
        "number": p.tag.number,
        "person": p.tag.person,
        "tense": p.tag.tense,
    }
    rus_rows.append(row)

rus_df = pd.DataFrame(rus_rows)

# Таблицы частот POS
eng_tokens_np = [t for t in word_tokenize(english_text) if re.search(r"[A-Za-z0-9]", t)]
eng_pos_counts = Counter(tag for _, tag in pos_tag(eng_tokens_np))
eng_freq_df = pd.DataFrame(eng_pos_counts.most_common(), columns=["pos_tag", "count"])

rus_pos_counts = Counter()
for word in rus_tokens:
    if all(not ch.isalnum() and ch not in "ёЁ" for ch in word):
        continue
    pos = morph.parse(word)[0].tag.POS or "UNKN"
    rus_pos_counts[pos] += 1

rus_freq_df = pd.DataFrame(rus_pos_counts.most_common(), columns=["pos_tag", "count"])

# Таблицы
print("ENG tagged table:")
print(eng_df)

print("\nRUS tagged table:")
print(rus_df)

print("\nENG POS frequency:")
print(eng_freq_df)

print("\nRUS POS frequency:")
print(rus_freq_df)

ENG tagged table:
       token pos_tag
0          ‘      JJ
1      Marty     NNP
2    Supreme     NNP
3          ’     NNP
4        Has     VBZ
..       ...     ...
418       of      IN
419   Jewish      JJ
420      New     NNP
421     York     NNP
422        .       .

[423 rows x 2 columns]

RUS tagged table:
           token pos_tag  case gender number person tense
0          Когда    CONJ  None   None   None   None  None
1    кавалеристы    NOUN  nomn   masc   plur   None  None
2      проходили    VERB  None   None   plur   None  past
3          через    PREP  None   None   None   None  None
4        деревню    NOUN  accs   femn   sing   None  None
..           ...     ...   ...    ...    ...    ...   ...
375       мордой    NOUN  ablt   femn   sing   None  None
376       из-под    PREP  None   None   None   None  None
377        снега    NOUN  gent   masc   sing   None  None
378          Иди    VERB  None   None   sing   None  None
379        копай    VERB  None   None   sing   No

**Задание 3: Синтаксический анализ предложений**

1. Возьмите 2 простых и 3 сложных предложений на русском и английском языке (всего 10 предложений).
2. Используйте SpaCy для построения синтаксических деревьев этих предложений.
3. Для каждого предложения:
* Визуализируйте синтаксическое дерево
* Выделите все подлежащие и сказуемые
* Найдите все пары слов, связанные отношением определения (прилагательное-существительное)
4. Разработайте функцию для извлечения всех объектных и субъектных отношений из предложения в формате (субъект, предикат, объект).
5. Объясните, какие трудности возникают при синтаксическом анализе сложных предложений

In [24]:
russian_sentences = [ "На лесной опушке уже слышатся птичьи голоса.",
"Звонко отстукивает клювом дятел.",
"Скоро солнце получше пригреет землю, почернеют дороги, обнажатся на полях проталины, зажурчат ручейки, пожалуют грачи.",
"Днём пригревало солнце, а ночью морозы доходили до пяти градусов.",
"Солнце только ещё поднималось , но его лучи уже освещали верхушки деревьев."]
english_sentences = [ "The cat is sleeping.", "She likes to read books.",
                     "Although it was raining, we decided to go for a walk in the park.",
                     "The book that I borrowed from the library is very interesting and has many useful facts.",
                     "Because he studied hard for the exam, he felt confident when he entered the classroom."]

In [25]:
!python -m spacy download ru_core_news_md
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 MB 17.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 31.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [26]:
# Загрузка языковых моделей
nlp_ru = spacy.load("ru_core_news_md")
nlp_en = spacy.load("en_core_web_md")

In [27]:
from spacy import displacy

In [28]:
# Визуализация дерева зависимостей для русского языка
for sentence in russian_sentences:
    doc_ru = nlp_ru(sentence)

    print(f"Sentence {russian_sentences.index(sentence)}")
    displacy.render(doc_ru, style="dep", jupyter=True, options={"distance": 100, "compact": True})

Sentence 0


Sentence 1


Sentence 2


Sentence 3


Sentence 4


In [29]:
# Визуализация дерева зависимостей для английского языка
for sentence in english_sentences:
    doc_en = nlp_en(sentence)

    print(f"Sentence {english_sentences.index(sentence)}")
    displacy.render(doc_en, style="dep", jupyter=True, options={"distance": 100, "compact": True})

Sentence 0


Sentence 1


Sentence 2


Sentence 3


Sentence 4


In [30]:
def find_subj_and_pred_eng(doc):
    subj = []
    pred_tokens = []
    root = None

    # Находим корень
    for token in doc:
        if token.dep_ == "ROOT":
            root = token
            pred_tokens.append(token)
            break

    if not root:
        return [], []

    # Находим все компоненты составного сказуемого
    for token in doc:
        if token.dep_ in ["aux", "auxpass"] and token.head in [root] + [t for t in pred_tokens]:
            pred_tokens.append(token)

    # Составляем сказуемое в правильном порядке
    pred = [doc[i].text for i in sorted([t.i for t in pred_tokens])]

    # Ищем подлежащее где угодно в предложении
    for token in doc:
        if token.dep_ in ["nsubj", "nsubjpass"]:
            # Проверяем, является ли это подлежащим по отношению к ROOT или его компонентам
            current = token
            while current.head != current:  # Поднимаемся по дереву до корня
                if current.head in pred_tokens or current.head == root:
                    # Нашли подлежащее, связанное с нашим сказуемым
                    subj.append(token.text)
                    break
                current = current.head

    # Если все равно не нашли подлежащее - ищем более агрессивно
    if not subj:
        for token in doc:
            if token.dep_ in ["nsubj", "nsubj:pass"]:
                subj.append(token.text)

    return subj, pred

In [31]:
print("Английские предложения")
for text in english_sentences:
    doc = nlp_en(text)
    subject, predicate = find_subj_and_pred_eng(doc)
    print(f"Предложение: {text}")
    print(f"Подлежащее: {subject}, Сказуемое: {predicate}")
    print()

Английские предложения
Предложение: The cat is sleeping.
Подлежащее: ['cat'], Сказуемое: ['is', 'sleeping']

Предложение: She likes to read books.
Подлежащее: ['She'], Сказуемое: ['likes']

Предложение: Although it was raining, we decided to go for a walk in the park.
Подлежащее: ['it', 'we'], Сказуемое: ['decided']

Предложение: The book that I borrowed from the library is very interesting and has many useful facts.
Подлежащее: ['book', 'I'], Сказуемое: ['is']

Предложение: Because he studied hard for the exam, he felt confident when he entered the classroom.
Подлежащее: ['he', 'he', 'he'], Сказуемое: ['felt']



In [32]:
def find_subj_pred_rus(doc):
    root = None
    pred_tokens = []

    for token in doc:
        if token.dep_ == "ROOT":
            root = token
            pred_tokens.append(token)
            break

    if not root:
        return [], []

    changed = True
    while changed:
        changed = False
        for t in doc:
            if t.dep_ == "conj" and t.head in pred_tokens and t.pos_ in {"VERB", "AUX"}:
                if t not in pred_tokens:
                    pred_tokens.append(t)
                    changed = True

    def pred_text(p):
        parts = {p}
        for ch in p.children:
            if ch.dep_ in {"aux", "auxpass", "cop", "neg"}:
                parts.add(ch)
            if ch.dep_ == "xcomp" and ch.pos_ in {"VERB", "AUX"}:
                parts.add(ch)
        return " ".join(tok.text for tok in sorted(parts, key=lambda x: x.i))

    def subjects(p):
        subs = [ch for ch in p.children if ch.dep_ in {"nsubj", "nsubjpass", "nsubj:pass"}]
        if subs:
            return subs
        if p.dep_ == "conj":
            return subjects(p.head)
        return []

    all_subj = []
    all_pred = []
    for p in sorted(pred_tokens, key=lambda x: x.i):
        all_pred.append(pred_text(p))
        all_subj.extend([s.text for s in subjects(p)])

    return all_subj, all_pred

In [33]:
print("Русские предложения")
for text in russian_sentences:
    doc = nlp_ru(text)
    subject, predicate = find_subj_pred_rus(doc)
    print(f"Предложение: {text}")
    print(f"Подлежащее: {subject}, Сказуемое: {predicate}")
    print()

Русские предложения
Предложение: На лесной опушке уже слышатся птичьи голоса.
Подлежащее: ['голоса'], Сказуемое: ['слышатся']

Предложение: Звонко отстукивает клювом дятел.
Подлежащее: ['Звонко'], Сказуемое: ['отстукивает']

Предложение: Скоро солнце получше пригреет землю, почернеют дороги, обнажатся на полях проталины, зажурчат ручейки, пожалуют грачи.
Подлежащее: ['солнце', 'солнце', 'солнце', 'солнце', 'грачи'], Сказуемое: ['пригреет', 'почернеют', 'обнажатся', 'зажурчат', 'пожалуют']

Предложение: Днём пригревало солнце, а ночью морозы доходили до пяти градусов.
Подлежащее: ['Днём', 'морозы'], Сказуемое: ['пригревало', 'доходили']

Предложение: Солнце только ещё поднималось , но его лучи уже освещали верхушки деревьев.
Подлежащее: ['Солнце', 'лучи'], Сказуемое: ['поднималось', 'освещали']



In [34]:
print("\nПоиск пар слов, связанных отношением определения (прилагательное-существительное) (русский):")
for sentence in russian_sentences:
    doc_ru = nlp_ru(sentence)
    for token in doc_ru:
        if token.dep_ == "amod":
            print(f"Найден объект: {token.text} — зависит от существительного: {token.head.text}")


Поиск пар слов, связанных отношением определения (прилагательное-существительное) (русский):
Найден объект: лесной — зависит от существительного: опушке
Найден объект: птичьи — зависит от существительного: голоса


In [35]:
print("\nПоиск пар слов, связанных отношением определения (прилагательное-существительное) (english):")
for sentence in english_sentences:
    doc_en = nlp_en(sentence)
    for token in doc_en:
        if token.dep_ == "amod":
            print(f"Найден объект: {token.text} — зависит от существительного: {token.head.text}")


Поиск пар слов, связанных отношением определения (прилагательное-существительное) (english):
Найден объект: many — зависит от существительного: facts
Найден объект: useful — зависит от существительного: facts


In [36]:
# Функция для извлечения всех объектных и субъектных отношений из предложения в формате (субъект, предикат, объект)
def get_svo(sentence, lang='en'):
    if lang == 'rus':
        doc = nlp_ru(sentence)
        for token in doc:
            if token.dep_ == "nsubj":
                subj = token.text
            if token.dep_ == "obj":
                obj = token.text
                predicate = token.head.text
                print(f"{subj} {predicate} {obj} ")
    else:
        doc = nlp_en(sentence)
        for token in doc:
            if token.dep_ in {"nsubj", "nsubjpass", "csubj", "csubjpass"}:
                subj = token.text
                predicate = token.head.text
            if token.dep_ in {"dobj", "iobj", "obj"}:
                obj = token.text
                print(f"{subj} {predicate} {obj}")

print("Русское SVO")
print(25 * '*')
for sentence in russian_sentences:
    get_svo(sentence, 'rus')

print("\nАнглийское SVO")
print(25 * '*')
for sentence in english_sentences:
    get_svo(sentence, 'en')

Русское SVO
*************************
Звонко отстукивает дятел 
солнце пригреет землю 
солнце почернеют дороги 
солнце зажурчат ручейки 
Днём пригревало солнце 
лучи освещали верхушки 

Английское SVO
*************************
She likes books
book is that
I borrowed facts
he entered classroom


**Задание 4: Распознавание именованных сущностей (Named Entity Recognition)**

1. Подготовьте корпус из 10 новостных текстов, содержащий различные типы именованных сущностей (имена людей, организации, географические названия, даты и т.д.) на английском или русском языке.
2. Используйте SpaCy для автоматического распознавания именованных сущностей.
3. Реализуйте свой простой метод для распознавания имен людей и географических названий с помощью регулярных выражений и словарей.
4. Сравните результаты работы SpaCy и вашего метода:
* Рассчитайте точность (precision), полноту (recall) и F1-меру для вашего метода относительно результатов SpaCy
* Проанализируйте ошибки обоих подходов, какие типы ошибок характерны для каждого подхода
5. Представьте сравнение результатов в виде таблицы

In [38]:
news_text = "news.txt"
# Reading text
with open(news_text, 'r', encoding='utf-8') as file:
    news_text = file.read()
print(f"Файл успешно прочитан. Длина текста: {len(news_text)} символов")

Файл успешно прочитан. Длина текста: 20470 символов


In [39]:
def spacy_ner(text, language='english'):
    """
    Распознавание именованных сущностей с помощью spaCy
    """

    doc = nlp_en(news_text)

    # Извлечение сущностей
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

entities_spacy = spacy_ner(text)
for entity, entity_type in entities_spacy:
        print(f"- {entity}: {entity_type}")

- Oscar: WORK_OF_ART
- 14: DATE
- only three: CARDINAL
- La La Land: WORK_OF_ART
- 2016: CARDINAL
- Titanic: ORG
- 1997: DATE
- All About Eve: WORK_OF_ART
- 1950: DATE
- the year: DATE
- 75-year-old: DATE
- two: CARDINAL
- year: DATE
- Thursday: DATE
- six: CARDINAL
- Oscar: WORK_OF_ART
- the Golden Globes: FAC
- Screen Actors Guild: ORG
- Producers Guild of America: ORG
- Directors Guild of America: ORG
- Five: CARDINAL
- this season: DATE
- Hamnet: ORG
- Marty Supreme: PERSON
- Frankenstein: PERSON
- half: CARDINAL
- Norwegian: NORP
- Sentimental Value: WORK_OF_ART
- Screen Actors Guild: ORG
- Stellan Skarsgard: PERSON
- the Golden Globe: FAC
- Oscar: PERSON
- Brazil: GPE
- The Secret Agent: WORK_OF_ART
- two: CARDINAL
- Globes: PERSON
- Iranian: NORP
- It Was Just An Accident: WORK_OF_ART
- Gotham: PERSON
- early December: DATE
- Palme d’Or: PERSON
- Cannes: NORP
- Netflix: ORG
- Jay Kelly: PERSON
- A House of Dynamite: WORK_OF_ART
- Train Dreams: WORK_OF_ART
- American Film Institu

In [54]:
import re
import string

date_patterns = [
    r"\b\d{1,2}[./-]\d{1,2}[./-]\d{2,4}\b",
    r"\b\d{4}\b"
]

money_patterns = [
    r"[$€£]\s?\d+(?:\.\d+)?"
]

percent_patterns = [
    r"\b\d+(?:\.\d+)?%\b"
]

email_patterns = [
    r"\b[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b"
]

url_patterns = [
    r"https?://[^\s]+"
]

phone_patterns = [
    r"\b\d{10,15}\b"
]

person_prefixes = {"Mr", "Mrs", "Ms", "Dr", "President", "Sir"}
organizations = {"UN", "NATO", "EU", "NASA", "Google", "Microsoft"}
location_prefixes = {"in", "at", "from"}

def rule_based(text):
    entities = []

    words = [w.strip(string.punctuation) for w in text.split()]

    known_people = {
        "Barack Obama", "Joe Biden", "Donald Trump", "Kamala Harris",
        "Angela Merkel", "Emmanuel Macron", "Justin Trudeau",
        "Narendra Modi", "Xi Jinping", "Volodymyr Zelenskyy",
        "Benjamin Netanyahu", "Pope Francis", "King Charles",
        "Albert Einstein", "Isaac Newton", "Marie Curie",
        "Taylor Swift", "Beyoncé", "Lionel Messi",
        "Cristiano Ronaldo", "Michael Jordan", "Serena Williams"
    }

    def safe_find(patterns, label):
        for p in patterns:
            try:
                for m in re.finditer(p, text):
                    entities.append((m.group(), label))
            except re.error:
                pass

    safe_find(date_patterns, "DATE")
    safe_find(money_patterns, "MONEY")
    safe_find(percent_patterns, "PERCENT")
    safe_find(email_patterns, "EMAIL")
    safe_find(url_patterns, "URL")
    safe_find(phone_patterns, "PHONE")

    # Контекстный поиск
    i = 0
    while i < len(words):
        word = words[i]

        # PERSON by prefix
        if word in person_prefixes and i + 1 < len(words):
            if words[i + 1][0].isupper():
                name = words[i] + " " + words[i + 1]
                entities.append((name, "PERSON"))
                i += 2
                continue

        # ORGANIZATION
        if word in organizations:
            entities.append((word, "ORGANIZATION"))

        # LOCATION
        if word in location_prefixes and i + 1 < len(words):
            if words[i + 1][0].isupper():
                entities.append((words[i + 1], "LOCATION"))

        i += 1

    for person in known_people:
        if person in text:
            entities.append((person, "FAMOUS"))

    return entities

rule_entities = rule_based(news_text)

print("\nEnglish parallel text:")
for e, t in rule_entities:
    print(f"- {e}: {t}")


English parallel text:
- 2016: DATE
- 1997: DATE
- 1950: DATE
- 2018: DATE
- 2002: DATE
- 2010: DATE
- 2011: DATE
- 2019: DATE
- 2018: DATE
- 2018: DATE
- 2027: DATE
- 2024: DATE
- 2020: DATE
- 2022: DATE
- 2024: DATE
- 2024: DATE
- $4.48: MONEY
- $480: MONEY
- $4.48: MONEY
- $300: MONEY
- $6.2: MONEY
- $12: MONEY
- $410: MONEY
- $0: MONEY
- $1: MONEY
- $0.20: MONEY
- $0.90: MONEY
- $1: MONEY
- $0.10: MONEY
- $10: MONEY
- $100: MONEY
- $1.4: MONEY
- $11: MONEY
- $9: MONEY
- Cannes: LOCATION
- Gmail: LOCATION
- Sir Lord: PERSON
- Danny: LOCATION
- Hollywood: LOCATION
- April: LOCATION
- Gmail: LOCATION
- Anthropic: LOCATION
- Google: ORGANIZATION
- Anthropic: LOCATION
- Ms Peng: PERSON
- Elon: LOCATION
- Google: LOCATION
- Google: ORGANIZATION
- Ms Peng: PERSON
- Silicon: LOCATION
- Google: ORGANIZATION
- Gmail: LOCATION
- Google: ORGANIZATION
- Mr Bezos: PERSON
- September: LOCATION
- Mr Zelikman: PERSON
- Mr Zelikman: PERSON
- Mr Zelikman: PERSON
- Mr Harik: PERSON
- Taylor: LOCATION

In [70]:
def evaluate_ner(predicted, gold_standard):
    """
    Оценка качества NER с помощью F-меры

    Аргументы:
        predicted: список кортежей (текст, тип) - предсказания модели
        gold_standard: список кортежей (текст, тип) - золотой стандарт

    Возвращает:
        precision, recall, f1: точность, полнота, F1-мера
    """
    # Преобразуем списки в множества для удобства сравнения
    predicted_set = set(predicted)
    gold_set = set(gold_standard)

    # Рассчитываем истинно положительные, ложно положительные и ложно отрицательные
    true_positives = len(predicted_set.intersection(gold_set))
    false_positives = len(predicted_set - gold_set)
    false_negatives = len(gold_set - predicted_set)

    # Рассчитываем точность, полноту и F1-меру
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

In [71]:
# Функция для вывода результатов оценки
def print_evaluation_results(method_name, precision, recall, f1):
    print(f"\nРезультаты для {method_name}:")
    print('-' * 50)
    print(f"Точность (Precision): {precision:.4f}")
    print(f"Полнота (Recall): {recall:.4f}")
    print(f"F1-мера: {f1:.4f}")

In [72]:
gold_standard = [
    ("TechCrunch Disrupt", "ORGANIZATION"),
    ("Jyoti Bansal", "PERSON"),
    ("Medha Agarwal", "PERSON"),
    ("Defy", "ORGANIZATION"),
    ("Jennifer Neundorfer", "PERSON"),
    ("January Ventures", "ORGANIZATION"),
    ("Disrupt 2026", "ORGANIZATION"),
    ("Google Cloud", "ORGANIZATION"),
    ("Netflix", "ORGANIZATION"),
    ("Microsoft", "ORGANIZATION"),
    ("Box", "ORGANIZATION"),
    ("Phia", "ORGANIZATION"),
    ("a16z", "ORGANIZATION"),
    ("ElevenLabs", "ORGANIZATION"),
    ("Wayve", "ORGANIZATION"),
    ("Hugging Face", "ORGANIZATION"),
    ("Elad Gil", "PERSON"),
    ("Vinod Khosla", "PERSON"),
    ("San Francisco", "LOCATION"),
    ("October 13-15, 2026", "DATE"),
    ("TechCrunch", "ORGANIZATION"),
    ("LTK CEO", "TITLE"),
    ("Amber Venz Box", "PERSON"),
    ("LTK", "ORGANIZATION"),
    ("Patreon CEO", "TITLE"),
    ("Jack Conte", "PERSON"),
    ("Patreon", "ORGANIZATION"),
    ("Northwestern University", "ORGANIZATION"),
    ("21%", "PERCENT"),
    ("97%", "PERCENT"),
    ("Sean Atkins", "PERSON"),
    ("CEO of short-form video production company Dhar Mann Studios", "TITLE"),
    ("Dhar Mann Studios", "ORGANIZATION"),
    ("Eric Wei", "PERSON"),
    ("cofounder of Karat Financial", "TITLE"),
    ("Karat Financial", "ORGANIZATION"),
    ("Discord", "ORGANIZATION"),
    ("Drake", "PERSON"),
    ("Kai Cenat", "PERSON"),
    ("top Twitch streamer", "TITLE"),
    ("Twitch", "ORGANIZATION"),
    ("Glenn Ginsburg", "PERSON"),
    ("president of QYOU Media", "TITLE"),
    ("QYOU Media", "ORGANIZATION"),
    ("Reed Duchscher", "PERSON"),
    ("founding CEO of Night", "TITLE"),
    ("Night", "ORGANIZATION"),
    ("MrBeast", "PERSON"),
    ("Merriam-Webster", "ORGANIZATION"),
    ("94%", "PERCENT"),
    ("Strava", "ORGANIZATION"),
    ("LinkedIn", "ORGANIZATION"),
    ("Substack", "ORGANIZATION"),
    ("PewDiePie", "PERSON"),
    ("Charli D’Amelio", "PERSON"),
    ("Alix Earle", "PERSON"),
    ("Outdoor Boys", "ORGANIZATION"),
    ("Epic Gardening", "ORGANIZATION"),
    ("YouTube", "ORGANIZATION"),
    ("United States", "LOCATION"),
    ("2025", "DATE"),
    ("OpenAI", "ORGANIZATION"),
    ("$40 billion", "MONEY"),
    ("$300 billion", "MONEY"),
    ("Safe Superintelligence", "ORGANIZATION"),
    ("Thinking Machine Labs", "ORGANIZATION"),
    ("$2 billion", "MONEY"),
    ("Meta", "ORGANIZATION"),
    ("$15 billion", "MONEY")
]

In [73]:
# Оценка rule-based NER
rule_precision, rule_recall, rule_f1 = evaluate_ner(rule_entities, gold_standard)

print_evaluation_results("Rule-based", rule_precision, rule_recall, rule_f1)


Результаты для Rule-based:
--------------------------------------------------
Точность (Precision): 0.0000
Полнота (Recall): 0.0000
F1-мера: 0.0000


In [74]:
# Оценка spaCy NER
spacy_precision, spacy_recall, spacy_f1 = evaluate_ner(entities_spacy, gold_standard)

print_evaluation_results("spaCy NER", spacy_precision, spacy_recall, spacy_f1)


Результаты для spaCy NER:
--------------------------------------------------
Точность (Precision): 0.0000
Полнота (Recall): 0.0000
F1-мера: 0.0000


In [75]:
import pandas as pd
# Таблица сравнения
results_df = pd.DataFrame([
    {
        "Method": "spaCy NER",
        "Precision": spacy_precision,
        "Recall": spacy_recall,
        "F1": spacy_f1,
    },
    {
        "Method": "Rule-based",
        "Precision": rule_precision,
        "Recall": rule_recall,
        "F1": rule_f1,
    }
])

results_df[["Precision","Recall","F1"]] = results_df[["Precision","Recall","F1"]].round(4)
results_df

,Method,Precision,Recall,F1
0,spaCy NER,0.0,0.0,0
1,Rule-based,0.0,0.0,0
